In [ ]:
import pandas as pd
import time
import dask
import os.path
import logging
logging.basicConfig(filename='out2/cluster_id_monitor')
import random

In [ ]:
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
opts = Options()
# opts.set_headless()
opts.headless = True
assert opts.headless  # Operating in headless mode
browser1 = Firefox(options=opts)
browser2 = Firefox(options=opts)
browser3 = Firefox(options=opts)
browser4 = Firefox(options=opts)
browsers = [browser1,browser2,browser3,browser4]

In [ ]:
def follow_link(cluster_id,browser):
    link = f"https://2019electionresults.comelec.gov.ph/#/search/{cluster_id}/local"
    browser.get(link)
    time.sleep(15)
    try:
        results = browser.find_elements_by_class_name('main-info-wrapper')
        return results[0] #c
    except:
        return

def get_transmission_status(c):
    try:
        transmission_status = c.find_element_by_class_name('transmission-status')
    except:
        return
    transmission_status_label = transmission_status.find_element_by_css_selector('.candidate-result-top').text
    transmission_status_text = transmission_status.find_element_by_xpath('/html/body/div/div/ui-view/div/div/div[2]/div[2]/div[2]/results-viewer/div[1]/div/div[2]').text
    d={transmission_status_label:transmission_status_text}
    df = pd.DataFrame([d])
    return df

def get_machine_info(c):
    machine_table = c.find_element_by_css_selector('html.ng-scope body.ng-scope div.content-wrapper div#container.container-fluid ui-view.ng-scope div.row.ng-scope div.form-group div.col-xs-12.col-sm-12.col-md-9.col-md-offset-3.results-info div.inside-full-height div.main-info-wrapper.col-xs-12.col-sm-12 results-viewer.ng-isolate-scope div.row.contests-wrapper.ng-scope div.ng-scope div.col-xs-12.col-sm-12.col-md-12.ng-scope')
    gen_info = machine_table.find_elements_by_class_name('gen-inf-row')
    infos = []
    for cc in gen_info:
        infos.append(cc.text.split('\n'))
    df_temp = pd.DataFrame(infos).T
    df = pd.DataFrame(df_temp.loc[1].values).T
    df.columns = df_temp.loc[0].values
    return df

def get_results_table(c):
    return c.find_elements_by_class_name('contest-results-table')

def get_senators(results_table):
    senators = []
    candidates = results_table[0].find_elements_by_xpath('/html/body/div/div/ui-view/div/div/div[2]/div[2]/div[2]/results-viewer/div[2]/div[2]/div[2]/div/div/div[2]/div[1]/*')
    for cc in candidates:
        senators.append(cc.text.split("\n"))
    df = pd.DataFrame(senators[1:])
    df.columns = senators[0]
    return df

def get_partylists(results_table):
    partylists = []
    candidates = results_table[2].find_elements_by_xpath('/html/body/div/div/ui-view/div/div/div[2]/div[2]/div[2]/results-viewer/div[2]/div[2]/div[3]/div/div/div[2]/div[1]/*')
    for cc in candidates:
        partylists.append(cc.text.split("\n"))
    df = pd.DataFrame(partylists[1:])
    df.columns = partylists[0]
    return df

def get_stats_senators(results_table):
    stats_senators = results_table[1].find_elements_by_xpath('/html/body/div/div/ui-view/div/div/div[2]/div[2]/div[2]/results-viewer/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/div/statistical-info/div/div[2]/*')
    stats = []
    for cc in stats_senators:
        stats.append(cc.text.split("\n"))
    df = pd.DataFrame(stats[1:])
    df.columns = ['Stat','Value']
    return df

def get_stats_partylists(results_table):
    stats_partylists = results_table[3].find_elements_by_xpath('/html/body/div/div/ui-view/div/div/div[2]/div[2]/div[2]/results-viewer/div[2]/div[2]/div[3]/div/div/div[2]/div[2]/div/statistical-info/div/div[2]/*')
    stats = []
    for cc in stats_partylists:
        stats.append(cc.text.split("\n"))
    df = pd.DataFrame(stats[1:])
    df.columns = ['Stat','Value']
    return df

@dask.delayed
def get_all_cluster_info_pipeline(cluster_id,browser):
    logging.info(f"RUN: {cluster_id, time.time()}")
    check = 0
    try:
        c = follow_link(cluster_id,browser)
    except:
        logging.warning(f"RERUN: {cluster_id, k, time.time()}")
        return
    
    if(c is not None):
        try:
            transmission_status = get_transmission_status(c)    
        except:
            logging.warning(f"RERUN: Transmission {cluster_id, time.time()}")
            return
        try:
            machine_info = get_machine_info(c)
            clustered_precinct = machine_info['Clustered precinct'].values[0]
        except:
            logging.warning(f"RERUN: Machine Info {cluster_id, time.time()}")
            return
        
        try:
            results_table = get_results_table(c)
        except:
            logging.warning(f"RERUN: Results DOM {cluster_id, time.time()}")
            return
        
        try:
            senators_votes = get_senators(results_table)
        except:
            logging.warning(f"RERUN: Senators {cluster_id, time.time()}")
            return
        
        try:
            senators_votes_stats = get_stats_senators(results_table)
        except:
            logging.warning(f"RERUN: Senators Stat {cluster_id, time.time()}")
            return
        
        try:
            partylists_votes = get_partylists(results_table)
        except:
            logging.warning(f"RERUN: Partylists {cluster_id, time.time()}")
            return
        
        try: 
            partylists_votes_stats = get_stats_partylists(results_table)
        except:
            logging.warning(f"RERUN: Partylists stat {cluster_id, time.time()}")
            return

        data = {'transmission_status':transmission_status, 'machine_info':machine_info, \
                'senators_votes': senators_votes, 'senators_votes_stats': senators_votes_stats, \
                'partylists_votes': partylists_votes, 'partylists_votes_stats': partylists_votes_stats}

        for k,v in data.items():
            v['cluster_id'] = cluster_id
            v['Clustered Precinct Extracted'] = clustered_precinct
            if(os.path.isfile(f"out2/{k}")==True):
                v.to_csv(f"out2/{k}", mode='a', header=False, index=False, encoding='utf-8')
            else:
                v.to_csv(f"out2/{k}", mode='w', index=False, encoding='utf-8')
            logging.info(f"OK: {cluster_id, k, time.time()}")

    else:
        logging.warning(f"RERUN: {cluster_id, time.time()}")
    return

In [ ]:
trans = pd.read_csv("out2/transmission_status")
mach = pd.read_csv("out2/machine_info")
mach.shape, trans.shape

In [ ]:
ccodes = pd.read_excel("./CCSCodes.xlsx")
p=pd.read_excel("./ProjectofPrecincts2019.xlsx")
clustered_precincts = p[['BALLOT_ID']].copy()
clustered_precincts['cluster_id'] = p['BALLOT_ID'].apply(lambda x: str(x).zfill(8))

In [ ]:
ncr=[ccc for ccc in pd.unique(ccodes['PROVINCE']) if 'NATIONAL CAPITAL REGION' in ccc ]
ncr_ccodes = ccodes[ccodes['PROVINCE'].isin(ncr)]['CCS_CODE'].values
ncr_ccodes = [str(ccc).zfill(4) for ccc in ncr_ccodes]

In [ ]:
clustered_precincts_filter = [ccc for ccc in clustered_precincts['cluster_id'].values if ccc[:4] in ncr_ccodes]

In [ ]:
%time
output = []
# browser = browser
skip_cluster_id = mach['Clustered precinct'].values.tolist()

for cluster_id in clustered_precincts_filter:
    if(cluster_id not in skip_cluster_id):
        task = get_all_cluster_info_pipeline(cluster_id,browsers[random.randint(0,3)])
    output.append(task)
    
all_data = dask.compute(*output)

#### Post-Processing

In [ ]:
senators = pd.read_csv("out2/senators_votes")
senators_stats = pd.read_csv("out2/senators_votes_stats")
partylists = pd.read_csv("out2/partylists_votes")
partylists_stats = pd.read_csv("out2/partylists_votes_stats")
machine_info = pd.read_csv("out2/machine_info")

machine_info = machine_info[machine_info['cluster_id']==machine_info['Clustered Precinct Extracted']].copy()
senators = senators[senators['cluster_id']==senators['Clustered Precinct Extracted']].copy()
partylists = partylists[partylists['cluster_id']==partylists['Clustered Precinct Extracted']].copy()

senators = senators.groupby(senators.columns.tolist()).nth(0).reset_index()
partylists = partylists.groupby(partylists.columns.tolist()).nth(0).reset_index()
machine_info = machine_info.groupby(machine_info.columns.tolist()).nth(0).reset_index()